<a href="https://colab.research.google.com/github/ymjo95/colab_tutorial2/blob/main/%EB%8D%B0%EC%9D%B4%ED%84%B0%EA%B1%B0%EB%B2%84%EB%84%8C%EC%8A%A4_%EC%8B%A4%EC%8A%B51%EB%AC%B8%EC%A0%9C(1).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd

# GitHub의 raw 파일 URL
url = "https://raw.githubusercontent.com/junhkang/manufacturing-data-governance-lab/main/data/sample_quality_data.xlsx"

!pip install openpyxl

# 파일 읽기
df = pd.read_excel(url, engine='openpyxl')
df.head()

,EQUIP_ID,TEMP_C,QUALITY_SCORE,INSPECT_DATE,STATUS
0,EQ002,259.4,105.0,NaN,NaN
1,EQ003,104.2,120.0,2024-04-02,WARN
2,EQ049,148.6,NaN,2024-04-03,WARN
3,EQ037,177.2,120.0,2024-04-04,OK
4,EQ049,117.2,73.0,2024-04-05,FAIL


In [ ]:
# 품질점수가 0~100점 사이여한다.
QUALITY_SCORE_RANGE = (0, 100)

# 온도는 -50~200도 사이가 정상 범위이다
TEMP_RANGE = (-50, 200)

# 상태코드는 OK/FAIL/WARN 중에 하나여야한다.
STATUS_ALLOWED = ["OK", "FAIL", "WARN"]

In [ ]:
df["TEMP_VALID"] = df["TEMP_C"].between(*TEMP_RANGE, inclusive="both")

df["QUALITY_VALID"] = df["QUALITY_SCORE"].between(*QUALITY_SCORE_RANGE, inclusive="both")

df["DATE_VALID"] = pd.to_datetime(df["INSPECT_DATE"], errors="coerce").notna()

df["STATUS_VALID"] = df["STATUS"].isin(STATUS_ALLOWED)

df[["TEMP_VALID","QUALITY_VALID","DATE_VALID","STATUS_VALID"]].head()

,TEMP_VALID,QUALITY_VALID,DATE_VALID,STATUS_VALID
0,False,False,False,False
1,True,False,True,True
2,True,False,True,True
3,True,False,True,True
4,True,True,True,True


In [ ]:
df["DATA_ISSUE"] = ~(
    df["TEMP_VALID"] &
    df["QUALITY_VALID"] &
    df["DATE_VALID"] &
    df["STATUS_VALID"]
)

df["FINAL_LABEL"] = df["DATA_ISSUE"].apply(lambda x:"FAIL" if x else "PASS")

df[["EQUIP_ID", "TEMP_C", "QUALITY_SCORE", "INSPECT_DATE", "STATUS", "FINAL_LABEL"]].head()


,EQUIP_ID,TEMP_C,QUALITY_SCORE,INSPECT_DATE,STATUS,FINAL_LABEL
0,EQ002,259.4,105.0,NaN,NaN,FAIL
1,EQ003,104.2,120.0,2024-04-02,WARN,FAIL
2,EQ049,148.6,NaN,2024-04-03,WARN,FAIL
3,EQ037,177.2,120.0,2024-04-04,OK,FAIL
4,EQ049,117.2,73.0,2024-04-05,FAIL,PASS


In [ ]:
df[df["DATA_ISSUE"] == True].head(10)

,EQUIP_ID,TEMP_C,QUALITY_SCORE,INSPECT_DATE,STATUS,TEMP_VALID,QUALITY_VALID,DATE_VALID,STATUS_VALID,DATA_ISSUE,FINAL_LABEL
0,EQ002,259.4,105.0,NaN,NaN,False,False,False,False,True,FAIL
1,EQ003,104.2,120.0,2024-04-02,WARN,True,False,True,True,True,FAIL
2,EQ049,148.6,NaN,2024-04-03,WARN,True,False,True,True,True,FAIL
3,EQ037,177.2,120.0,2024-04-04,OK,True,False,True,True,True,FAIL
6,EQ049,130.4,NaN,2024-04-07,OK,True,False,True,True,True,FAIL
7,EQ002,151.6,40.0,NaN,FAIL,True,True,False,True,True,FAIL
8,EQ002,48.3,105.0,2024-04-09,FAIL,True,False,True,True,True,FAIL
9,EQ028,999.9,40.0,2024-04-10,OK,False,True,True,True,True,FAIL
11,EQ037,123.4,NaN,2024-04-12,WARN,True,False,True,True,True,FAIL
12,EQ032,156.9,NaN,2024-04-13,FAIL,True,False,True,True,True,FAIL


In [ ]:
issue_summary = df[df["DATA_ISSUE"] == True].groupby("EQUIP_ID").size().reset_index(name="ISSUE_COUNT")
issue_summary = issue_summary.sort_values(by="ISSUE_COUNT", ascending=False)
issue_summary.head(10)

,EQUIP_ID,ISSUE_COUNT
29,EQ032,9
45,EQ048,9
1,EQ002,6
34,EQ037,6
32,EQ035,6
13,EQ016,5
23,EQ026,5
21,EQ024,5
20,EQ023,5
16,EQ019,5
